In [ ]:
import sympy

from scipy.interpoate import interp1d

In [ ]:
def generate_polynomial(data, degree):
    ''' Makes a random polynomial
    
        data: The data as a numeric value to be split
        degree: The degree of the polynomial, and the minimum number of keys needed to decrypt
        
        Returns: polynomial as a function
    '''
    
    pass

def generate_keys(n, poly):
    ''' Creates the keys
    
        n: Number of keys to be created
        poly: Polynomial function to create the keys
        
        Returns: n-tuple of keys
    '''
    
    pass

def decrypt(keys):
    ''' Uses Lagrange interpolation to decrypt data
    
        keys: List of keys to use in the dectyption
        
        Returns: Decrypted data
    '''
    
    pass
